In [1]:
import spacy
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#nltk.download('hedgewords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#from nltk.corpus import hedge_words
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spradha5\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spradha5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
data = pd.read_csv("P2_Training_Dataset - Copy.csv") 
post=data['text']
ide=data['thread_id']
label=data['delta']

In [4]:
def cosine(s1,s2):
    s1_vec = nlp(s1)
    s2_vec = nlp(s2)
    return s1_vec.similarity(s2_vec)

In [5]:
analyser = SentimentIntensityAnalyzer()
def sentiment(s2):
    score = analyser.polarity_scores(s2)
    listscore = list(score.values())
    return listscore

In [6]:
xtrain=[]
ytrain=[]

#print(type(post.loc[3]))
#print(len(post.loc[3]))
op=0;
for i in range(1,ide.size):
    if ide.loc[i]!=ide.loc[op]:
        op=i
        continue
    s1=str(post.loc[op])
    s2=str(post.loc[i])
    lab=label.loc[i]
    
    f1=len(s2)
    
    #print((s2))
    f2=cosine(s1,s2)
    f3=sentiment(s2)
    #f4=hedge(s2)
    
    vec=[f1,f2]
    xtrain.append(vec + f3)
    ytrain.append(lab)


C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [14]:
xtrain=np.array(xtrain).astype(np.int32)
ytrain=np.array(ytrain).astype(np.int32)



(822,)

In [7]:


rfc = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features='auto')
rfc.fit(xtrain, ytrain)
print('Random Forest percentage score on 25% validation=',rfc.score(xtrain,ytrain)*100)

Random Forest percentage score on 25% validation= 99.7566909975669


In [8]:
print(ytrain[0])

0


In [9]:
datat = pd.read_csv("P2_Testing_Dataset - Copy.csv") 
postt=datat['text']
idet=datat['thread_id']
labelt=datat['delta']

In [10]:
xtest=[]
ytest=[]

#print(type(post.loc[3]))
#print(len(post.loc[3]))
opt=0;
for i in range(1,idet.size):
    if idet.loc[i]!=idet.loc[opt]:
        opt=i
        continue
    s1t=str(postt.loc[opt])
    s2t=str(postt.loc[i])
    labt=labelt.loc[i]
    
    f1=len(s2t)
    
    #print((s2))
    f2=cosine(s1t,s2t)
    f3=sentiment(s2t)
    #f4=hedge(s2)
    
    vec=[f1,f2]
    xtest.append(vec + f3)
    ytest.append(labt)


C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [15]:
ypred = rfc.predict(xtest)
print(rfc.score(xtest,ytest)*100)
print(type(ypred))

98.91146589259797
<class 'numpy.ndarray'>


In [12]:
#Performance Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('\n Confusion Matrix: \n', confusion_matrix(ytest,ypred,labels=[0,1]))
print('\n Accuracy: ', accuracy_score(ytest,ypred)*100)
print('\n Precision, Recall, F1-score: \n',classification_report(ytest,ypred))


 Confusion Matrix: 
 [[1363    0]
 [  15    0]]

 Accuracy:  98.91146589259797


C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 Precision, Recall, F1-score: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      1363
           1       0.00      0.00      0.00        15

    accuracy                           0.99      1378
   macro avg       0.49      0.50      0.50      1378
weighted avg       0.98      0.99      0.98      1378

